In [1]:
import numpy as np
import pandas as pd
import csv
import json
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from numpy.linalg import norm
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [2]:
def cosineSim(x, y):
    x_ = x/norm(x)
    y_ = y/norm(y)
    return(np.dot(x_, y_))

In [3]:
# Preprocess
ps = PorterStemmer()

with open('/content/sample_data/stopwords_new.txt') as file:
    stop = file.read()
stopwords = stop.split('\n')

num = list(range(3000))
num = [str(x) for x in num]
stopwords = stopwords + num

def removeStopwords(text):
    for word in text:
        if word in stopwords:
            text.remove(word)
    return text

def tokenize(text):
    token = [] #store token
    sub = "" # store temporary str
    for i in text:
        if i !=" ":
            if i in (",",".","'","?",'"',"@","-","$","!","_","`","(","","*","&","/",")","","''"):
                continue
            sub+=i
            if sub == "\n": #\n 是txt的文字格式，在處理token時就刪除
                sub=""
                continue
        else:
            token.append(sub)
            sub=""
    if sub :
        token.append(sub)
    return token

def preprocessing(text):
#     tokens = [word for word in nltk.word_tokenize(text)]
    tokens = tokenize(text)
    stems = [ps.stem(item) for item in tokens]
    final = []
    return stems

In [4]:
# read in text
with open("/content/sample_data/textWords.csv", 'r') as f:
    reader = csv.reader(f)
    textWords = np.array([row for row in reader], dtype=object)

In [5]:
# train doc2vec
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(textWords)]
model = Doc2Vec(documents, dm=0, vector_size=100, window=10, min_count=1, workers=4, epochs=100)

### Query testing

In [6]:
%%time
temp = preprocessing(input())
query_vector = model.infer_vector(temp)

war
CPU times: user 39.1 ms, sys: 9.59 ms, total: 48.7 ms
Wall time: 3.5 s


In [7]:
%%time
result = []
for ind in range(1000):
    result.append([cosineSim(query_vector, model.dv[ind]), ind])
sorted_result = sorted(result, key=lambda x: x[0], reverse=True)

CPU times: user 15.7 ms, sys: 0 ns, total: 15.7 ms
Wall time: 16.4 ms


In [8]:
series_titles = []

with open("/content/sample_data/imdb_top_1000.csv", 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        # 將 Series_Title (假設在第一列) 加入列表
        series_titles.append(row[1])

returned_movies = []
for index, item in enumerate(sorted_result):
    if index >= 5:
        break
    if(sorted_result[index][0]!=0):
        movie_name = series_titles[item[1]]
        movie_Correlation = sorted_result[index][0]
        returned_movies.append(movie_name)
        print(movie_name,  movie_Correlation)

Zwartboek 0.8350779
Patton 0.82679796
The Pianist 0.80062926
Das Boot 0.7971175
Kaze tachinu 0.7952487


In [9]:
# for checking movie
df = pd.read_csv('/content/sample_data/imdb_top_1000_preprocessing.csv')
df_movie = df[["Series_Title", "Overview"]]

In [10]:
for i, movie in enumerate(returned_movies):
  print(i, ". ", df_movie.iloc[df.index[df_movie.Series_Title == movie][0]][1])

0 .  In the Nazi-occupied Netherlands during World War II, a Jewish singer infiltrates the regional Gestapo headquarters for the Dutch resistance.
1 .  The World War II phase of the career of controversial American general George S. Patton.
2 .  A Polish Jewish musician struggles to survive the destruction of the Warsaw ghetto of World War II.
3 .  The claustrophobic world of a WWII German U-boat; boredom, filth and sheer terror.
4 .  A look at the life of Jiro Horikoshi, the man who designed Japanese fighter planes during World War II.


In [11]:
# K = int(input()) # check movie
# print("Series_Title: ", df_movie.iloc[K][0])
# print("Overview: ", df_movie.iloc[K][1])